<a href="https://colab.research.google.com/github/taegukang35/ML-study/blob/main/LIME_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import gray2rgb, rgb2gray
from skimage.util import montage as montage2d
from sklearn.datasets import fetch_olivetti_faces

In [ ]:
faces = fetch_olivetti_faces()

X_vec = np.stack([gray2rgb(iimg) for iimg in faces.data.reshape((-1,64,64))],0)
y_vec =  faces.target.astype(np.uint8)

In [ ]:
fig,ax1 = plt.subplots(1,1,figsize=(8,8))
ax1.imshow(montage2d(X_vec[:,:,:,0]),cmap='gray',interpolation='none')
ax1.set_title('All Faces')
ax1.axis('off')

In [ ]:
y_vec

In [ ]:
index = 105
plt.imshow(X_vec[index],cmap='gray')
plt.title('{} index face'.format(index))
plt.axis('off')

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X_vec,y_vec,train_size=0.70)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.neural_network import MLPClassifier

class PipeStep(object):
    def __init__(self,step_func):
        self.__step_func=step_func
    def fit(self,*args):
        return self
    def transform(self,X):
        return self.__step_func(X)

In [ ]:
# (400x64x64x3) 데이터를 흑백으로 바꿈-> RGB 3채널을 1채널로 합산 (400x64x64x1)
makegray_step = PipeStep(lambda img_list: [rgb2gray(img) for img in img_list])
# 64x64 크기의 이미지를 한 줄로 펼쳐준다 64x64 = 4096 (MLP 적용하기 위함)
flatten_step = PipeStep(lambda img_list: [img.ravel() for img in img_list])
simple_pipeline = Pipeline([
                            ('Make Gray',makegray_step),
                            ('Flatten Image',flatten_step),
                            ('Normalize',Normalizer()),
                            ('MLP',MLPClassifier(
                                activation='relu',
                                alpha = 1e-7,
                                epsilon = 1e-6,
                                hidden_layer_sizes=(800,120),
                                random_state=1))
                            ])

In [ ]:
simple_pipeline.fit(X_train,y_train)

In [ ]:
pipe_pred_test = simple_pipeline.predict(X_test)
pipe_pred_prop = simple_pipeline.predict_proba(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test,y_pred=pipe_pred_test))

In [ ]:
!pip install lime

In [ ]:
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm

explainer = lime_image.LimeImageExplainer()

# 이미지 분할 알고리즘: quickshift, slick, felzenszwalb
segmenter = SegmentationAlgorithm('slic',n_segments =100,compactness=1,sigma=1)

In [ ]:
olivetti_test_index = 0
exp = explainer.explain_instance(X_test[olivetti_test_index],
                                 classifier_fn=simple_pipeline.predict_proba,
                                 top_labels = 6,
                                 num_samples = 1000,
                                 segmentation_fn=segmenter)

In [ ]:
from skimage.color import label2rgb
# 캔버스 설정
fig,((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize=(8,8))

# 예측에 도움이 되는 세그먼트 출력 
temp,mask = exp.get_image_and_mask(y_test[olivetti_test_index],positive_only=True,num_features=8,hide_rest=False)
ax1.imshow(label2rgb(mask,temp,bg_label=0),interpolation='nearest')
ax1.set_title('Positive Region for {}'.format(y_test[olivetti_test_index]))

# 모든 세그먼트 출력
temp,mask = exp.get_image_and_mask(y_test[olivetti_test_index],positive_only=False,num_features=8,hide_rest=False)
ax2.imshow(label2rgb(4-mask,temp,bg_label=0),interpolation='nearest')
ax2.set_title('Positive/Negative Regions for {}'.format(y_test[olivetti_test_index]))

# 이미지만 출력
ax3.imshow(temp,interpolation='nearest')
ax3.set_title('Show output image only')

# 마스크만 출력
ax4.imshow(mask,interpolation='nearest')
ax4.set_title('Show mask only')